In [2]:
### Retrieving Insider buy side transactions and storing it in a list of lists 

import pandas as pd 

buying_url = "http://insidertrading.org/large-stock-buying.php?sort_by=acceptance_datetime&asc=&symbol=&date_from=2015-08-01&date_to=2016-08-07&submit=+GO+&page="
data = []
url_pages = []

def extract_table(url):    
    buying_text = requests.get(url).text
    buying_parsed = BeautifulSoup(buying_text)

    buying_table = buying_parsed.find("table", {"id": "tracker"})
    buying = buying_table.find("tbody")
    rows = buying.find_all("tr")

    for row in rows:
        cols = row.find_all('td')
        cols = [item.text.strip() for item in cols]
        data.append([item for item in cols])

url_pages = [buying_url+str(i+1) for i in range(0,64)]    
map(extract_table, url_pages)
    
## Checking I have the expected data set
print len(data)
df = pd.DataFrame(data)
print df.tail()

19124
        0           1                    2   \
19119  Buy  2015-07-29  2015-08-03 12:21:04   
19120  Buy  2015-07-31  2015-08-03 12:10:14   
19121  Buy  2015-07-31  2015-08-03 11:47:12   
19122  Buy  2015-07-31  2015-08-03 08:15:13   
19123  Buy  2015-07-31  2015-08-03 08:00:11   

                                      3     4                   5   \
19119                          Point.360  PTSX   BAGERDJIAN HAIG S   
19120                  NeuroMetrix, Inc.  NURO         Gozani Shai   
19121                  Aqua Metals, Inc.  AQMS          Slade Mark   
19122  AIR PRODUCTS & CHEMICALS INC /DE/   APD       Ghasemi Seifi   
19123                           LKQ CORP  LKQX  Zarcone Dominick P   

                               6       7          8                9   \
19119  director officer 10% owner  20,000    $0.6900     $13,800.0000   
19120            director officer  17,857    $0.8366     $14,939.2000   
19121                    director  20,000    $5.0000    $100,000.0000  

In [3]:
### Adding Insider selling side transactions to the data

selling_url = "http://insidertrading.org/large-stock-selling.php?sort_by=acceptance_datetime&asc=&symbol=&date_from=2015-08-01&date_to=2016-08-07&submit=+GO+&page="
url_pages = [selling_url+str(i+1) for i in range(0,71)]    
     
map(extract_table, url_pages)
    
print len(data)
df = pd.DataFrame(data)

print df.tail()

40267
         0           1                    2                               3   \
40262  Sell  2015-07-24  2015-08-03 13:04:31    SUNOCO LOGISTICS PARTNERS LP   
40263  Sell  2015-07-31  2015-08-03 11:36:14         CHEMICAL FINANCIAL CORP   
40264  Sell  2015-08-01  2015-08-03 10:25:47  ALBANY INTERNATIONAL CORP /DE/   
40265  Sell  2015-07-15  2015-08-03 10:06:51                       YSEEK INC   
40266  Sell  2015-07-15  2015-08-03 10:06:51                       YSEEK INC   

         4                      5         6        7         8   \
40262   SXL      Lauterbach Kurt A   officer   24,550  $34.7200   
40263  CHFC    Lievense Richard M.  director   11,188  $32.9069   
40264   AIN  Halftermeyer Daniel A   officer   21,311  $39.0500   
40265  ATDU           Phipps David            200,000   $0.8000   
40266  ATDU           Phipps David             50,000   $0.1000   

                  9        10      11  
40262  $852,376.0000  131,854  Form 4  
40263  $368,162.0000   74,097 

In [4]:
#Renaming Columns
df.columns= ["Side", "Date", "AcceptanceDate", "Company", "Symbol", "Owner", "OwnerRel", "Shares", "SharePrice", "Value", "Remaining", "Form"]
df.head()

,Side,Date,AcceptanceDate,Company,Symbol,Owner,OwnerRel,Shares,SharePrice,Value,Remaining,Form
0,Buy,2016-08-04,2016-08-05 20:31:38,RESMED INC,RMD,FARRELL PETER C,director officer,"20,000",$25.5400,"$510,800.0000","367,978",Form 4
1,Buy,2016-08-04,2016-08-05 20:29:23,ENTEGRIS INC,ENTG,GMT CAPITAL CORP,10% owner,"1,191,220",$16.8000,"$20,012,500.0000","14,525,400",Form 4
2,Buy,2016-08-03,2016-08-05 20:07:56,DXP ENTERPRISES INC,dxpe,GMT CAPITAL CORP,10% owner,"172,746",$15.9600,"$2,757,030.0000","1,595,540",Form 4
3,Buy,2016-08-05,2016-08-05 20:04:56,SALESFORCE COM INC,CRM,Benioff Marc,director officer 10% owner,"12,500",$27.0625,"$338,281.0000","35,383,000",Form 4
4,Buy,2016-08-05,2016-08-05 19:46:39,FAIR ISAAC CORP,FIC,Wehmann James M,officer,"16,215",$43.9000,"$711,838.0000","64,611",Form 4


In [97]:
df.to_csv("/Users/sharanduggal/Documents/Datascience/Projects/WebScrapingProjects/Insider/Past_Year_Transactions", sep = "\t")

In [10]:
###Getting ticker's sector and industry details from Finviz 
symbol_pages = "http://finviz.com/quote.ashx?t=bac&ty=c&ta=1&p=d"
print symbol_pages
sector = []

def get_industry(symbol_pages):
    count = 0
    stock_url = requests.get(symbol_pages)
    stock_text = stock_url.text
    stock_soup = BeautifulSoup(stock_text)
    stock_table = stock_soup.find("table", attrs={"class": "fullview-title"}) 
    ###Creating a list of sector & industry
    ###Inserting a blank record in the event a ticker symbol is wrong or if a ticker page is not found on Finviz
    if stock_url.ok: 
        sector.append([i.text for i in stock_table.find_all("a")[0:4]]) 
        count = count+1
        if count%2 == 0:
            print count
    else:
        sector.append(["", "", "", ""])

http://finviz.com/quote.ashx?t=bac&ty=c&ta=1&p=d


In [8]:
### Test Cell
x = df.head()
Urls = "http://finviz.com/quote.ashx?t="+x.Symbol+"&ty=c&ta=1&p=d"
map(get_industry, Urls) 
print sector

#ind_df = pd.DataFrame(sector)
#ind_df.columns = ["Symbol", "Company", "Sector", "Industry"]
#del ind_df["Company"]
#newdf = pd.concat([x,ind_df], axis = 1)


[[u'RMD', u'ResMed Inc.', u'Healthcare', u'Medical Appliances & Equipment'], [u'ENTG', u'Entegris, Inc.', u'Technology', u'Semiconductor Equipment & Materials'], [u'DXPE', u'DXP Enterprises, Inc.', u'Services', u'Industrial Equipment Wholesale'], [u'CRM', u'salesforce.com, inc.', u'Technology', u'Application Software'], ['', '', '', '']]


In [11]:
##Creating Finviz URLs for each symbol in the data set
Urls = "http://finviz.com/quote.ashx?t="+df.Symbol+"&ty=c&ta=1&p=d"
print "End"

End


In [ ]:
###Applying function to all Urls
map(get_industry, Urls) 
print "End"

In [101]:
###Concat the new Finviz Info with the Original data frame
ind_df = pd.DataFrame(sector)
ind_df.columns = ["Symbol", "Company", "Sector", "Industry"]
del ind_df["Company"]

newdf = pd.concat([df,ind_df], axis = 1)

[[u'RMD', u'ResMed Inc.', u'Healthcare', u'Medical Appliances & Equipment'], [u'ENTG', u'Entegris, Inc.', u'Technology', u'Semiconductor Equipment & Materials'], [u'DXPE', u'DXP Enterprises, Inc.', u'Services', u'Industrial Equipment Wholesale'], [u'CRM', u'salesforce.com, inc.', u'Technology', u'Application Software'], ['', '', '', '']]


,Side,Date,AcceptanceDate,Company,Symbol,Owner,OwnerRel,Shares,SharePrice,Value,Remaining,Form,Symbol,Sector,Industry
0,Buy,2016-08-04,2016-08-05 20:31:38,RESMED INC,RMD,FARRELL PETER C,director officer,"20,000",$25.5400,"$510,800.0000","367,978",Form 4,RMD,Healthcare,Medical Appliances & Equipment
1,Buy,2016-08-04,2016-08-05 20:29:23,ENTEGRIS INC,ENTG,GMT CAPITAL CORP,10% owner,"1,191,220",$16.8000,"$20,012,500.0000","14,525,400",Form 4,ENTG,Technology,Semiconductor Equipment & Materials
2,Buy,2016-08-03,2016-08-05 20:07:56,DXP ENTERPRISES INC,dxpe,GMT CAPITAL CORP,10% owner,"172,746",$15.9600,"$2,757,030.0000","1,595,540",Form 4,DXPE,Services,Industrial Equipment Wholesale
3,Buy,2016-08-05,2016-08-05 20:04:56,SALESFORCE COM INC,CRM,Benioff Marc,director officer 10% owner,"12,500",$27.0625,"$338,281.0000","35,383,000",Form 4,CRM,Technology,Application Software
4,Buy,2016-08-05,2016-08-05 19:46:39,FAIR ISAAC CORP,FIC,Wehmann James M,officer,"16,215",$43.9000,"$711,838.0000","64,611",Form 4,,,


In [ ]:
newdf.head()

In [ ]:
newdf.tail()

In [ ]:
df.to_csv("/Users/sharanduggal/Documents/Datascience/Projects/WebScrapingProjects/Insider/Insiders_Aug8", sep = "\t")

1
